In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 讀入深度學習套件

In [3]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.datasets import imdb

## 讀入數據

In [4]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 0s 0us/step


In [5]:
len(x_train)

25000

In [6]:
len(x_test)

25000

In [8]:
len(x_train[0])

218

In [9]:
len(x_train[1])

189

In [12]:
y_train[0] #positive commit

1

In [13]:
y_train[1] #negative commit

0

## 資料處理

In [15]:
x_train = sequence.pad_sequences(x_train, maxlen=100) #剪成100字
x_test = sequence.pad_sequences(x_train, maxlen=100)

## 打造一個函數學習機

In [17]:
model = Sequential()

In [18]:
model.add(Embedding(10000, 128)) #壓成幾維，10000壓成128維

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [20]:
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2)) #LSTM不一定要配合前面的數字
#drop out指定隨機多少百分比的先不用學習，dropout可有可無

ValueError: Input 0 of layer lstm_1 is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: [None, 128]

In [21]:
model.add(Dense(1, activation='sigmoid'))

In [22]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1280000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,411,713
Trainable params: 1,411,713
Non-trainable params: 0
_________________________________________________________________


In [26]:
(128+128+1)*4*128

131584

## 訓練

In [29]:
model.fit(x_train, y_train, batch_size=32, epochs=10, 
          validation_data=(x_test, y_test)) #看測試資料的誤差

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 236s 9ms/sample - loss: 0.4566 - acc: 0.7900 - val_loss: 0.9963 - val_acc: 0.4993
Epoch 2/10
25000/25000 [==============================] - 232s 9ms/sample - loss: 0.3215 - acc: 0.8660 - val_loss: 1.2384 - val_acc: 0.4983
Epoch 3/10
25000/25000 [==============================] - 242s 10ms/sample - loss: 0.2518 - acc: 0.9002 - val_loss: 1.7054 - val_acc: 0.4948
Epoch 4/10
25000/25000 [==============================] - 235s 9ms/sample - loss: 0.1950 - acc: 0.9248 - val_loss: 2.4212 - val_acc: 0.4949
Epoch 5/10
25000/25000 [==============================] - 241s 10ms/sample - loss: 0.1552 - acc: 0.9405 - val_loss: 2.2681 - val_acc: 0.4969
Epoch 6/10
25000/25000 [==============================] - 231s 9ms/sample - loss: 0.1212 - acc: 0.9566 - val_loss: 2.8435 - val_acc: 0.4973
Epoch 7/10
25000/25000 [==============================] - 220s 9ms/sample - loss: 0.0947 - acc: 0.9666 - val

In [30]:
model_json = model.to_json()
open('imdb_model_architecture.json', 'w').write(model_json)
model.save_weights('imdb_model_weights.h5')